# Data Extraction

* This is an updated version of 'DSCI511 data extraction text_v2', streamlined for clarity and readibility 
* Jillian put this notebook together, and drew from 'DSCI511 data extraction text' and 'Raw Data Filtering' by Ryan 
* In this notebook, we read in the JSON data that we saved from the full API call, extract key parameters, examine as a dataframe, and save as a TSV

### Imports and Installs

In [30]:
import json
from pprint import pprint
import zipfile
import pandas as pd

### Read in JSON data from zip file

In [7]:
#This code block from Ryan

#The path to the zipfile
zip_path = "./raw data/raw_launch_data.json.zip" #Ryan had "../raw data/raw_launch_data.json.zip", but this only worked for me with one period
launch_data_filename = 'raw_launch_data.json'

#loading the raw launch data 
with zipfile.ZipFile(zip_path, "r") as z:
    with z.open(launch_data_filename) as f:
        raw_launch_data = json.load(f)

In [17]:
#Check data type is correct, what are the keys, look at the metadata
print(type(raw_launch_data))
pprint(raw_launch_data.keys())
print()
print(f"Collector: {raw_launch_data['collector']}, Total launches: {raw_launch_data['total_launches']}, "
    "Collection date: {raw_launch_data['collection_date']}")
print("Total number of launches:", len(raw_launch_data['launches']))

<class 'dict'>
dict_keys(['collector', 'total_launches', 'collection_date', 'launches'])

Collector: RyanPtest, Total launches: 7336, Collection date: {raw_launch_data['collection_date']}
Total number of launches: 7336


### Select data to use 
In code below, select whether to use a smaller sample of the data or the full data. Each item in the list within `raw_launch_data["launches"]` is an individual launch.

In [67]:
#Save a smaller data sample to test:
#data_sample = raw_launch_data["launches"][:100]  #[-100:]

#If you want the full data instead, comment out the above and un-comment this:
data_sample = raw_launch_data["launches"]

### Create a table in a data loop

For each launch dictionary, grab the specific information that we want and add it to a table (list of lists)

Make sure to include the tagID, to potentially be able to link dataframes

#### - Launch table (to be updated)

In [77]:
#Referencing lecture notes 4.1.2
launch_table = [] #intialize

#TODO: These headers and the data gathered in loop subject to change based on what we decide we want!
launch_header = ["ID", "Date_of_Launch", "Lauch_Status", "Short_Form_Status", "Launch_pad_location_name", "Launch_pad_name", "Launch_pad_country_name",
   "Launch_pad_country_ID", "Launch_pad_lattitude", "Launch_pad_longitude"]
#launch_table.append(header)

#TODO: we could also do some preliminary cleaning in this loop if we want, or save it for later
for launch in data_sample:
    ID = launch['id']
    
    Date = launch['net']
    Status = launch['status']['name']
    SF_status = launch['status']['abbrev']
    Pad_location = launch['pad']['location']['name']
    Pad_name = launch['pad']['name']
    
    #The following cause the error 'NoneType object is not subscriptable' - added code to address, but should double check this is correct
    if launch['pad']['country']: 
        Pad_country_name = launch['pad']['country']['name']
        Pad_country_ID = launch['pad']['country']['id']
    else:
        Pad_country_name = None
        Pad_country_ID = None
    
    Latt = launch['pad']['latitude']
    Long = launch['pad']['longitude']
    #TODO: maybe add reason for failure, if there is one - launch['failreason']

    row = [ID, Date, Status, SF_status, Pad_location, Pad_name, Pad_country_name, Pad_country_ID, Latt, Long]
    launch_table.append(row)

#### - Rocket table
#### TODO: Decide if we want any additional params or to omit any params. A lot of the payload-related fields seem to always be empty, so decide if we want to hunt for anything payload-related as well

In [117]:
rocket_table = [] #intialize

#TODO: These headers and the data gathered in loop subject to change based on what we decide we want!
rocket_header = ["ID", "Rocket_name", "Manufacturer_name", "Manufacturer_country", "Manufacturer_company_type", "Launch_service_provider",
                 "Reusability", "Min_no_stages", "Max_no_stages", "Rocket_length", "Rocket_diameter", "Launch_cost", "Liftoff_mass_tons", 
                 "Liftoff_thrust_kN", "Rocket_apogee"]
#rocket_table.append(header)

#TODO: we could also do some preliminary cleaning in this loop if we want, or save it for later
for launch in data_sample:
    ID = launch['id']

    rocket_name = launch['rocket']['configuration']['full_name'] #could also just do 'name'

    #The below cause 'list index out of range' errors - adding code to just fill with None for now
    if len(launch['rocket']['configuration']['families']) > 0:
        manuf_name = launch['rocket']['configuration']['families'][0]['manufacturer'][0]['name']
        manuf_country = launch['rocket']['configuration']['families'][0]['manufacturer'][0]['country'][0]['name']
        manuf_type = launch['rocket']['configuration']['families'][0]['manufacturer'][0]['type']['name']
    else: 
        manuf_name = None
        manuf_country = None
        manuf_type = None

    provider = launch["launch_service_provider"]["name"]
    reuse = launch['rocket']['configuration']['reusable']
    min_stage = launch['rocket']['configuration']['min_stage'] 
    max_stage = launch['rocket']['configuration']['max_stage'] 
    length = launch['rocket']['configuration']['length']
    diameter = launch['rocket']['configuration']['diameter']
    cost = launch['rocket']['configuration']['launch_cost']
    mass = launch['rocket']['configuration']['launch_mass']
    thrust = launch['rocket']['configuration']['to_thrust']
    apogee = launch['rocket']['configuration']['apogee'] #Need to figure out what unit is used for apogee

    #Things I did not include, but we could if we want:
    #replaced rocket_name = launch['rocket']['configuration']['name'] with "full_name"
    #print("Company Founding Year : ", data_sample[-1]['rocket']['configuration']['families'][0]['manufacturer'][0]['founding_year'])
    #print("Rocket Payload Mass to LEO (kg): ", data_sample[-1]['rocket']['configuration']['leo_capacity'])
    #print("Rocket Payload Mass to GTO (kg): ", data_sample[-1]['rocket']['configuration']['gto_capacity'])
    #print("Rocket Payload Mass to GEO (kg): ", data_sample[-1]['rocket']['configuration']['geo_capacity'])  #Could be removed if all null
    #print("Rocket Payload Mass to SSO (kg): ", data_sample[-1]['rocket']['configuration']['sso_capacity'])  #Could be removed if all null
    
    row = [ID, rocket_name, manuf_name, manuf_country, manuf_type, provider, reuse, min_stage, max_stage, length, diameter, cost, mass, thrust, apogee]
    rocket_table.append(row)

### Create Pandas dataframes

In [118]:
rocket_dataframe = pd.DataFrame(rocket_table, columns = rocket_header)
rocket_dataframe.tail()

,ID,Rocket_name,Manufacturer_name,Manufacturer_country,Manufacturer_company_type,Launch_service_provider,Reusability,Min_no_stages,Max_no_stages,Rocket_length,Rocket_diameter,Launch_cost,Liftoff_mass_tons,Liftoff_thrust_kN,Rocket_apogee
7331,5fc609f3-4aee-4ffd-968d-b4cddcd9e381,Long March 7A,China Aerospace Science and Technology Corpora...,China,Government,China Aerospace Science and Technology Corpora...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7332,7afcacb9-32aa-41ee-a000-0c7158f324c2,Ariane 62,Arianespace,France,Commercial,Arianespace,False,2.0,2.0,63.0,5.40,85000000.0,530.0,10370.0,NaN
7333,5d816773-89cb-48b9-9bf0-a9c4d8236785,Electron,None,None,None,Rocket Lab,False,2.0,3.0,18.0,1.20,6000000.0,13.0,162.0,NaN
7334,9dd2d2b7-302b-4e9e-804c-aa176f606b6f,Falcon 9 Block 5,SpaceX,United States of America,Commercial,SpaceX,True,1.0,2.0,70.0,3.65,52000000.0,549.0,7607.0,200.0
7335,6602c88f-cbff-4495-b417-a184ddb0a426,Falcon 9 Block 5,SpaceX,United States of America,Commercial,SpaceX,True,1.0,2.0,70.0,3.65,52000000.0,549.0,7607.0,200.0


In [119]:
#Referenced Stack Overflow on how to turn list of lists into Pandas dataframe: 
#https://stackoverflow.com/questions/32857544/nested-list-to-pandas-dataframe-with-headers

launch_dataframe = pd.DataFrame(launch_table, columns = launch_header)
launch_dataframe.head()

,ID,Date_of_Launch,Lauch_Status,Short_Form_Status,Launch_pad_location_name,Launch_pad_name,Launch_pad_country_name,Launch_pad_country_ID,Launch_pad_lattitude,Launch_pad_longitude
0,e3df2ecd-c239-472f-95e4-2b89b4f75800,1957-10-04T19:28:34Z,Launch Successful,Success,"Baikonur Cosmodrome, Republic of Kazakhstan",1/5,Kazakhstan,44.0,45.9200,63.342000
1,f8c9f344-a6df-4f30-873a-90fe3a7840b3,1957-11-03T02:30:00Z,Launch Successful,Success,"Baikonur Cosmodrome, Republic of Kazakhstan",1/5,Kazakhstan,44.0,45.9200,63.342000
2,535c1a09-97c8-4f96-bb64-6336d4bcb1fb,1957-12-06T16:44:35Z,Launch Failure,Failure,"Cape Canaveral SFS, FL, USA",Launch Complex 18A,United States of America,2.0,28.4493,-80.564494
3,1b9e28d0-c531-44b0-9b37-244e62a6d3f4,1958-02-01T03:47:56Z,Launch Successful,Success,"Cape Canaveral SFS, FL, USA",Launch Complex 26A,United States of America,2.0,28.4433,-80.571200
4,48bc7deb-b2e1-46c2-ab63-0ce00fbd192b,1958-02-05T07:33:00Z,Launch Failure,Failure,"Cape Canaveral SFS, FL, USA",Launch Complex 18A,United States of America,2.0,28.4493,-80.564494


### Null values 
### TODO: add better way to handle nulls in Rocket_dataframe, decide how to handle nulls in Launch_dataframe
The list of null values in the rocket dataframe was the same whether I looked at first 100 or last 100 launches. \
For now, I'm just filling any null values with "unknown"

In [120]:
rocket_dataframe.isnull().any()

ID                           False
Rocket_name                  False
Manufacturer_name             True
Manufacturer_country          True
Manufacturer_company_type     True
Launch_service_provider      False
Reusability                  False
Min_no_stages                 True
Max_no_stages                 True
Rocket_length                 True
Rocket_diameter               True
Launch_cost                   True
Liftoff_mass_tons             True
Liftoff_thrust_kN             True
Rocket_apogee                 True
dtype: bool

In [121]:
launch_dataframe.isnull().any()

ID                          False
Date_of_Launch              False
Lauch_Status                False
Short_Form_Status           False
Launch_pad_location_name    False
Launch_pad_name             False
Launch_pad_country_name      True
Launch_pad_country_ID        True
Launch_pad_lattitude        False
Launch_pad_longitude        False
dtype: bool

In [122]:
#Fill nulls in rocket_dataframe with 'Unknown', since for now there's not anything else to go on
rocket_dataframe_filled = rocket_dataframe.fillna("Unknown")
rocket_dataframe_filled.isnull().any()

ID                           False
Rocket_name                  False
Manufacturer_name            False
Manufacturer_country         False
Manufacturer_company_type    False
Launch_service_provider      False
Reusability                  False
Min_no_stages                False
Max_no_stages                False
Rocket_length                False
Rocket_diameter              False
Launch_cost                  False
Liftoff_mass_tons            False
Liftoff_thrust_kN            False
Rocket_apogee                False
dtype: bool

In [84]:
rocket_dataframe_filled.head()

,ID,Rocket_name,Manufacturer_name,Manufacturer_country,Manufacturer_company_type,Reusability,Min_no_stages,Max_no_stages,Rocket_length,Rocket_diameter,Launch_cost,Liftoff_mass_tons,Liftoff_thrust_kN,Rocket_apogee
0,e3df2ecd-c239-472f-95e4-2b89b4f75800,Sputnik 8K74PS,Energia,Russia,Government,False,1.0,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,f8c9f344-a6df-4f30-873a-90fe3a7840b3,Sputnik 8K74PS,Energia,Russia,Government,False,1.0,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,535c1a09-97c8-4f96-bb64-6336d4bcb1fb,Vanguard,Unknown,Unknown,Unknown,False,3.0,3.0,23.0,1.14,Unknown,10.0,123.0,Unknown
3,1b9e28d0-c531-44b0-9b37-244e62a6d3f4,Juno-I,Chrysler,United States of America,Commercial,False,4.0,4.0,21.2,1.78,Unknown,29.0,416.0,Unknown
4,48bc7deb-b2e1-46c2-ab63-0ce00fbd192b,Vanguard,Unknown,Unknown,Unknown,False,3.0,3.0,23.0,1.14,Unknown,10.0,123.0,Unknown


### TODO: Merge dataframes

In [65]:
#Add stuff here

### Save data as a TSV
### TODO: add code to save straight from Pandas dataframe to TSV

In [22]:
#Since launch_pad_location_name has commas, not ideal to save as a CSV - saving it as a TSV instead
#TODO: add a header?
with open("data_test/launch_table_test.tsv", "w") as f: 
    for row in launch_table:

        #Everything in the TSV needs to be a string, so do that conversion here:
        row_string = []
        for element in row: 
            if type(element) != str: 
                element_string = str(element)
                row_string.append(element_string)
            else:
                row_string.append(element) 

        #Now actually write to the tsv:
        f.write("\t".join(row_string) + "\n") #joining elements together w/ a tab

-----

### Below - Test Data Extraction
Feel free to reference the below, or to ignore

In [23]:
data_sample[0].keys()

dict_keys(['id', 'url', 'name', 'response_mode', 'slug', 'launch_designator', 'status', 'last_updated', 'net', 'net_precision', 'window_end', 'window_start', 'image', 'infographic', 'probability', 'weather_concerns', 'failreason', 'hashtag', 'launch_service_provider', 'rocket', 'mission', 'pad', 'webcast_live', 'program', 'orbital_launch_attempt_count', 'location_launch_attempt_count', 'pad_launch_attempt_count', 'agency_launch_attempt_count', 'orbital_launch_attempt_count_year', 'location_launch_attempt_count_year', 'pad_launch_attempt_count_year', 'agency_launch_attempt_count_year', 'flightclub_url', 'updates', 'info_urls', 'vid_urls', 'timeline', 'pad_turnaround', 'mission_patches'])

In [61]:
print(data_sample[0]['window_start'])
print(data_sample[0]['id'])
print()
pprint(data_sample[0]['rocket'])

1957-10-04T19:28:34Z
e3df2ecd-c239-472f-95e4-2b89b4f75800

{'configuration': {'active': False,
                   'alias': '',
                   'apogee': None,
                   'attempted_landings': 0,
                   'consecutive_successful_landings': 0,
                   'consecutive_successful_launches': 2,
                   'description': 'An early Russian rocket designed by Sergei '
                                  'Korolev in the Soviet Union',
                   'diameter': None,
                   'failed_landings': 0,
                   'failed_launches': 0,
                   'families': [{'active': False,
                                 'attempted_landings': 0,
                                 'consecutive_successful_landings': 0,
                                 'consecutive_successful_launches': 3,
                                 'description': '',
                                 'failed_landings': 0,
                                 'failed_launches': 1,
    

### Data Structure (Ryan, Jillian updates)

The data is a list of dictionaries that detail the information based on the rocket. From my search it appears that each disctionary is related a a certain rocket. Here are my initial findings from looking at how the data is structured.

* Within each launch dictionary there are multiple keys.
    - Some keys behave as key value pairs
    - Some keys have dictionaries as their values, so they may have further nested levels

## Feature Navigation

In this section I aim to locate specific features we wish to access to create our own dataset


### The Rocket

1. Rocket Name, Specific Variant Used
2. Company that Conducted the launch
3. Model Type
4. Paylod

For the example we will be using the first launch

In [29]:
pprint(data_sample[0]['rocket']['configuration']['families'][0]['manufacturer'][0])

{'abbrev': 'OKB-1',
 'administrator': None,
 'attempted_landings': 0,
 'attempted_landings_payload': 0,
 'attempted_landings_spacecraft': 0,
 'consecutive_successful_landings': 0,
 'consecutive_successful_launches': 0,
 'country': [{'alpha_2_code': 'RU',
              'alpha_3_code': 'RUS',
              'id': 5,
              'name': 'Russia',
              'nationality_name': 'Russian',
              'nationality_name_composed': 'Russo'}],
 'description': None,
 'failed_landings': 0,
 'failed_landings_payload': 0,
 'failed_landings_spacecraft': 0,
 'failed_launches': 0,
 'featured': False,
 'founding_year': 1946,
 'id': 1000,
 'image': None,
 'info_url': None,
 'launchers': '',
 'logo': None,
 'name': 'Energia',
 'parent': None,
 'pending_launches': 0,
 'response_mode': 'normal',
 'social_logo': None,
 'social_media_links': [],
 'spacecraft': '',
 'successful_landings': 0,
 'successful_landings_payload': 0,
 'successful_landings_spacecraft': 0,
 'successful_launches': 0,
 'total_laun

In [62]:
# Rocket Name and Company Name
print("Rocket Name : ", data_sample[0]['rocket']['configuration']['name'])
print("Company Name : ", data_sample[0]['rocket']['configuration']['families'][0]['manufacturer'][0]['name'])
print("Company Founding Year : ", data_sample[0]['rocket']['configuration']['families'][0]['manufacturer'][0]['founding_year'])
print("Country Affiliation : ", data_sample[0]['rocket']['configuration']['families'][0]['manufacturer'][0]['country'][0]['name'])
print("Company Type : ", data_sample[0]['rocket']['configuration']['families'][0]['manufacturer'][0]['type']['name'])

Rocket Name :  Sputnik 8K74PS
Company Name :  Energia
Company Founding Year :  1946
Country Affiliation :  Russia
Company Type :  Government


In [31]:
data_sample[0]['failreason']

''

In [63]:
# Launch related parameters
print("Launch related parameters")
print("---------------")
print("Date of Launch : ", data_sample[0]['net'])
print("Lauch Status : ", data_sample[0]['status']['name'])
print("Short Form Status:", data_sample[0]['status']['abbrev'])
print("Launch pad location name:", data_sample[0]['pad']['location']['name'])
print("Launch pad name:", data_sample[0]['pad']['name'])
print("Launch pad country name:", data_sample[0]['pad']['country']['name'])
print("Launch pad country name:", data_sample[0]['pad']['country']['id'])
print("Launch pad lattitude:", data_sample[0]['pad']['latitude'])
print("Launch pad longitude:", data_sample[0]['pad']['longitude'])

print()

Launch related parameters
---------------
Date of Launch :  1957-10-04T19:28:34Z
Lauch Status :  Launch Successful
Short Form Status: Success
Launch pad location name: Baikonur Cosmodrome, Republic of Kazakhstan
Launch pad name: 1/5
Launch pad country name: Kazakhstan
Launch pad country name: 44
Launch pad lattitude: 45.92
Launch pad longitude: 63.342



In [85]:
### Rocket Data - Ryan

# Rocket Name and Company Name
print("Rocket Name : ", data_sample[-1]['rocket']['configuration']['name'])
print("Company Name : ", data_sample[-1]['rocket']['configuration']['families'][0]['manufacturer'][0]['name'])
print("Company Founding Year : ", data_sample[-1]['rocket']['configuration']['families'][0]['manufacturer'][0]['founding_year'])
print("Country Affiliation : ", data_sample[-1]['rocket']['configuration']['families'][0]['manufacturer'][0]['country'][0]['name'])
print("Company Type : ", data_sample[-1]['rocket']['configuration']['families'][0]['manufacturer'][0]['type']['name'])

#Rocket Related Parameters
print("Is the Rocket Reusable : ", data_sample[-1]['rocket']['configuration']['reusable'])
print("Min. No. Stages : ", data_sample[-1]['rocket']['configuration']['min_stage'])
print("Max No. Stages : ", data_sample[-1]['rocket']['configuration']['max_stage'])
print("Rocket Length (meters): ", data_sample[-1]['rocket']['configuration']['length'])
print("Rocket Diameter (meters): ", data_sample[-1]['rocket']['configuration']['diameter'])
print("Rocket Launch Cost : ", data_sample[-1]['rocket']['configuration']['launch_cost'])
print("Rocket Liftoff Mass (Tons): ", data_sample[-1]['rocket']['configuration']['launch_mass'])
print("Rocket Payload Mass to LEO (kg): ", data_sample[-1]['rocket']['configuration']['leo_capacity'])
print("Rocket Payload Mass to GTO (kg): ", data_sample[-1]['rocket']['configuration']['gto_capacity'])
print("Rocket Payload Mass to GEO (kg): ", data_sample[-1]['rocket']['configuration']['geo_capacity'])  #Could be removed if all null
print("Rocket Payload Mass to SSO (kg): ", data_sample[-1]['rocket']['configuration']['sso_capacity'])  #Could be removed if all null
print("Rocket Liftoff Thrust (kN): ", data_sample[-1]['rocket']['configuration']['to_thrust'])
print("Rocket Apogee: ", data_sample[-1]['rocket']['configuration']['apogee']) #Need to figure out what unit is used for apogee

Rocket Name :  Falcon 9
Company Name :  SpaceX
Company Founding Year :  2002
Country Affiliation :  United States of America
Company Type :  Commercial
Is the Rocket Reusable :  True
Min. No. Stages :  1
Max No. Stages :  2
Rocket Length (meters):  70.0
Rocket Diameter (meters):  3.65
Rocket Launch Cost :  52000000
Rocket Liftoff Mass (Tons):  549.0
Rocket Payload Mass to LEO (kg):  22800.0
Rocket Payload Mass to GTO (kg):  8300.0
Rocket Payload Mass to GEO (kg):  None
Rocket Payload Mass to SSO (kg):  None
Rocket Liftoff Thrust (kN):  7607.0
Rocket Apogee:  200.0


In [94]:
#Rocket data - Jillian additions
print(data_sample[-1]['rocket']['configuration']["full_name"])
print()

print(data_sample[-1]['rocket']['configuration'].keys())

Falcon 9 Block 5

dict_keys(['response_mode', 'id', 'url', 'name', 'families', 'full_name', 'variant', 'active', 'is_placeholder', 'manufacturer', 'program', 'reusable', 'image', 'info_url', 'wiki_url', 'description', 'alias', 'min_stage', 'max_stage', 'length', 'diameter', 'maiden_flight', 'launch_cost', 'launch_mass', 'leo_capacity', 'gto_capacity', 'geo_capacity', 'sso_capacity', 'to_thrust', 'apogee', 'total_launch_count', 'consecutive_successful_launches', 'successful_launches', 'failed_launches', 'pending_launches', 'attempted_landings', 'successful_landings', 'failed_landings', 'consecutive_successful_landings', 'fastest_turnaround'])


In [97]:
print(data_sample[-1]["launch_service_provider"]["name"])
print()

print(data_sample[-1]["launch_service_provider"].keys())

SpaceX

dict_keys(['response_mode', 'id', 'url', 'name', 'abbrev', 'type', 'featured', 'country', 'description', 'administrator', 'founding_year', 'launchers', 'spacecraft', 'parent', 'image', 'logo', 'social_logo', 'total_launch_count', 'consecutive_successful_launches', 'successful_launches', 'failed_launches', 'pending_launches', 'consecutive_successful_landings', 'successful_landings', 'failed_landings', 'attempted_landings', 'successful_landings_spacecraft', 'failed_landings_spacecraft', 'attempted_landings_spacecraft', 'successful_landings_payload', 'failed_landings_payload', 'attempted_landings_payload', 'info_url', 'wiki_url', 'social_media_links'])


In [111]:
print(data_sample[-1].keys())
print()
#print(data_sample[-1]["program"]) #.keys())

dict_keys(['id', 'url', 'name', 'response_mode', 'slug', 'launch_designator', 'status', 'last_updated', 'net', 'net_precision', 'window_end', 'window_start', 'image', 'infographic', 'probability', 'weather_concerns', 'failreason', 'hashtag', 'launch_service_provider', 'rocket', 'mission', 'pad', 'webcast_live', 'program', 'orbital_launch_attempt_count', 'location_launch_attempt_count', 'pad_launch_attempt_count', 'agency_launch_attempt_count', 'orbital_launch_attempt_count_year', 'location_launch_attempt_count_year', 'pad_launch_attempt_count_year', 'agency_launch_attempt_count_year', 'flightclub_url', 'updates', 'info_urls', 'vid_urls', 'timeline', 'pad_turnaround', 'mission_patches'])



In [106]:
print(data_sample[-1]["mission"]['description'])

A batch of 28 satellites for the Starlink mega-constellation - SpaceX's project for space-based Internet communication system.


In [116]:
print(data_sample[-1]["launch_service_provider"]["name"])

SpaceX


In [138]:
print(data_sample[-1]["rocket"].keys())
print(data_sample[-1]["rocket"]["payloads"]) #This seems to always be empty

dict_keys(['id', 'configuration', 'launcher_stage', 'spacecraft_stage', 'payloads'])
[]


In [148]:
print(data_sample[-1]["rocket"]["configuration"].keys()) #["spacecraft_stage"]["docking_events"]["payload"])
print()
print(data_sample[5]["rocket"]["spacecraft_stage"]) #.keys())

dict_keys(['response_mode', 'id', 'url', 'name', 'families', 'full_name', 'variant', 'active', 'is_placeholder', 'manufacturer', 'program', 'reusable', 'image', 'info_url', 'wiki_url', 'description', 'alias', 'min_stage', 'max_stage', 'length', 'diameter', 'maiden_flight', 'launch_cost', 'launch_mass', 'leo_capacity', 'gto_capacity', 'geo_capacity', 'sso_capacity', 'to_thrust', 'apogee', 'total_launch_count', 'consecutive_successful_launches', 'successful_launches', 'failed_launches', 'pending_launches', 'attempted_landings', 'successful_landings', 'failed_landings', 'consecutive_successful_landings', 'fastest_turnaround'])

[]
